# entity special token 전처리 결과

In [3]:
import pickle as pickle
import os
import pandas as pd
import torch
import numpy as np
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, Trainer, TrainingArguments, RobertaConfig, RobertaTokenizer, RobertaForSequenceClassification, BertTokenizer
import re

train_data = pd.read_csv("../../data/train/train.csv")

## 기존 entity 분리 코드

In [4]:
def extract_entity(
    input_df: pd.DataFrame, tokenizer=None, drop_column=False
) -> pd.DataFrame:
    """
    데이터셋 DataFrame을 입력받고, entity의 ['word', 'start_idx', 'end_idx', 'type'] 요소들을 추출하여 새 column으로 생성합니다.
    추가되는 새 column들은 다음과 같습니다.
        subject_word, subejct_start_idx, subject_end_idx, subject_type
        object_word, object_start_idx, object_end_idx, object_type

    parameters
        tokenizer : transformers.AutoTokenizer를 입력하면 subject_word, object_word가 토큰으로 분리되어 처리됩니다.
        drop_column : 기존 subject_entity, object_entity column들을 drop할지 여부를 결정합니다.
    """
    df = input_df.copy()
    col_list = ["word", "start_idx", "end_idx", "type"]
    sub_contents = [[] for _ in range(len(col_list))]
    obj_contents = [[] for _ in range(len(col_list))]

    for sub_items, obj_items in zip(df["subject_entity"], df["object_entity"]):
        # 쉼표 중 앞에는 문자가 있고, 뒤에는 공백이 있으며 공백 뒤에 \' 가 있는 쉼표를 기준으로 split합니다.
        sub_items = re.split(r"(?<=\S),\s(?=\')", sub_items[1:-1])
        obj_items = re.split(r"(?<=\S),\s(?=\')", obj_items[1:-1])
        for i, (sub_content, obj_content) in enumerate(zip(sub_items, obj_items)):
            # ':' 중 앞에는 \' 가 있고, 뒤에는 공백이 있는 ':'를 기준으로 split합니다.
            sub_key, sub_insert = map(
                str.strip, re.split(r"(?<=\'):(?=\s)", sub_content)
            )
            obj_key, obj_insert = map(
                str.strip, re.split(r"(?<=\'):(?=\s)", obj_content)
            )
            # 문자열의 맨 처음에 위치한 \' 와 맨 뒤에 위치한 \'를 제거합니다.
            sub_insert = re.sub("^'|'$", "", sub_insert)
            obj_insert = re.sub("^'|'$", "", obj_insert)
            if tokenizer and sub_key == "'word'":
                sub_contents[i].append(tokenizer.tokenize(str(sub_insert)))
                obj_contents[i].append(tokenizer.tokenize(str(obj_insert)))
            else:
                sub_contents[i].append(sub_insert)
                obj_contents[i].append(obj_insert)

    # entity의 elements들을 새 column으로 추가합니다.
    prefix_list = ["subject_", "object_"]
    for prefix, contents in zip(prefix_list, [sub_contents, obj_contents]):
        for col, content in zip(col_list, contents):
            col_name = prefix + col
            df[col_name] = content
            if "idx" in col_name:
                df[col_name] = df[col_name].astype("int")
    if drop_column:
        df = df.drop(["subject_entity", "object_entity"], axis=1)
    return df

dataset = extract_entity(train_data, drop_column=True)
dataset.head(5)

,id,sentence,label,source,subject_word,subject_start_idx,subject_end_idx,subject_type,object_word,object_start_idx,object_end_idx,object_type
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,no_relation,wikipedia,비틀즈,24,26,ORG,조지 해리슨,13,18,PER
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,no_relation,wikitree,민주평화당,19,23,ORG,대안신당,14,17,ORG
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,org:member_of,wikitree,광주FC,21,24,ORG,한국프로축구연맹,34,41,ORG
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,org:top_members/employees,wikitree,아성다이소,13,17,ORG,박정부,22,24,PER
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,no_relation,wikipedia,요미우리 자이언츠,22,30,ORG,1967,0,3,DAT


## entity token 전처리 코드

Dataframe을 탐색할 때 iterrows로 했다가 1분이상 걸려서 numpy를 이용하여 최적화했습니다.

In [5]:
def entity_tagging(
    dataset: pd.DataFrame, 
) -> pd.DataFrame:
    """
    sentence에 entity type token을 추가하여 전처리합니다.
    
    -  〈Something〉는 <obj:PER>조지 해리슨</obj:PER>이 쓰고 <subj:ORG>비틀즈</subj:ORG>가 1969년 앨범
    """
    df = dataset.copy()
    data_np = dataset.to_numpy() # numpy로 데이터 탐색
    data_np = np.transpose(data_np)
    
    # data_np의 인덱스입니다
    # 0:'id', 1:'sentence', 2:'label', 3:'source', 4:'subject_word', 5:'subject_start_idx',
    # 6:'subject_end_idx', 7:'subject_type', 8:'object_word', 9:'object_start_idx',
    # 10:'object_end_idx', 11:'object_type')

    for i in df.index:
        subject_start_marker = f"<subj:{data_np[7][i]}>"
        subject_end_marker   = f"</subj:{data_np[7][i]}>"
        object_start_marker  = f"<obj:{data_np[11][i]}>"
        object_end_marker    = f"</obj:{data_np[11][i]}>"
        
        if "'" in data_np[8][i]: # (')이 포함된 문장 전처리
            data_np[8][i] = data_np[8][i].strip('"')
            
        sent = data_np[1][i]
        
        if data_np[5][i] < data_np[9][i]:  # subject가 object보다 앞에 있는 겨우
            tmp = (sent[0:data_np[5][i]] + subject_start_marker + data_np[4][i] + subject_end_marker +
                   sent[data_np[6][i]+1:data_np[9][i]] + object_start_marker + data_np[8][i] + object_end_marker + sent[data_np[10][i]+1:-1])
            
            data_np[5][i] += len(subject_start_marker) # subject_start_idx
            data_np[6][i] += len(subject_start_marker) # subject_end_idx
            data_np[9][i] += len(subject_start_marker + subject_end_marker + object_start_marker) # object_start_idx
            data_np[10][i] += len(subject_start_marker + subject_end_marker + object_start_marker) # object_end_idx
            
        else:  # object가 subject보다 앞에 있는 경우
            tmp = (sent[0:data_np[9][i]] + object_start_marker + data_np[8][i] + object_end_marker +
                   sent[data_np[10][i]+1:data_np[5][i]] + subject_start_marker + data_np[4][i] + subject_end_marker + sent[data_np[6][i]+1:-1])

            data_np[5][i] += len(object_start_marker + object_end_marker + subject_start_marker) # subject_start_idx
            data_np[6][i] += len(object_start_marker + object_end_marker + subject_start_marker) # subject_end_idx
            data_np[9][i] += len(object_start_marker) # object_start_idx
            data_np[10][i] += len(object_start_marker) # object_end_idx 
            
        str_tmp = "".join(tmp)
        data_np[1][i] = str_tmp # sentence
        
    df["sentence"] = data_np[1].copy()
    df["subject_start_idx"] = data_np[5].copy()
    df["subject_end_idx"] = data_np[6].copy()
    df["object_start_idx"] = data_np[9].copy()
    df["object_end_idx"] = data_np[10].copy()
    
    return df

%time output = entity_tagging(dataset)

CPU times: user 572 ms, sys: 12 ms, total: 584 ms
Wall time: 585 ms


## 코드 테스트

### 전처리 결과(sentence, idx 변경)

In [6]:
output.head(3)

,id,sentence,label,source,subject_word,subject_start_idx,subject_end_idx,subject_type,object_word,object_start_idx,object_end_idx,object_type
0,0,〈Something〉는 <obj:PER>조지 해리슨</obj:PER>이 쓰고 <su...,no_relation,wikipedia,비틀즈,53,55,ORG,조지 해리슨,22,27,PER
1,1,호남이 기반인 바른미래당·<obj:ORG>대안신당</obj:ORG>·<subj:OR...,no_relation,wikitree,민주평화당,48,52,ORG,대안신당,23,26,ORG
2,2,K리그2에서 성적 1위를 달리고 있는 <subj:ORG>광주FC</subj:ORG>...,org:member_of,wikitree,광주FC,31,34,ORG,한국프로축구연맹,64,71,ORG


### sentence 결과

In [7]:
pd.set_option("max_colwidth",300)
output["sentence"][0:3]

0                                            〈Something〉는 <obj:PER>조지 해리슨</obj:PER>이 쓰고 <subj:ORG>비틀즈</subj:ORG>가 1969년 앨범 《Abbey Road》에 담은 노래다
1                                                 호남이 기반인 바른미래당·<obj:ORG>대안신당</obj:ORG>·<subj:ORG>민주평화당</subj:ORG>이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다
2    K리그2에서 성적 1위를 달리고 있는 <subj:ORG>광주FC</subj:ORG>는 지난 26일 <obj:ORG>한국프로축구연맹</obj:ORG>으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다
Name: sentence, dtype: object

### 변경된 idx 확인

변경된 idx를 슬라이스로 검색했습니다.

In [8]:
for i in range(len(output)):
    sub_word = output["subject_word"][i]
    ob_word = output["object_word"][i].strip('"') if "'" in output["object_word"][i] else output["object_word"][i]
    
    if sub_word != output["sentence"][i][output["subject_start_idx"][i]:output["subject_end_idx"][i]+1]:
        print("sub_error","idx :",i)
    if ob_word != output["sentence"][i][output["object_start_idx"][i]:output["object_end_idx"][i]+1]:
        print("ob_error","idx :",i)        
        

## tokenizer 테스트

In [9]:
TYPE_TOKENS = dict(
    subject_per_start="<subj:PER>",
    subject_org_start="<subj:ORG>",
    subject_per_end="</subj:PER>",
    subject_org_end="</subj:ORG>",
    object_per_start="<obj:PER>",
    object_org_start="<obj:ORG>",
    object_loc_start="<obj:LOC>",
    object_dat_start="<obj:DAT>",
    object_poh_start="<obj:POH>",
    object_noh_start="<obj:NOH>",
    object_per_end="</obj:PER>",
    object_org_end="</obj:ORG>",
    object_loc_end="</obj:LOC>",
    object_dat_end="</obj:DAT>",
    object_poh_end="</obj:POH>",
    object_noh_end="</obj:NOH>",
)

MODEL_NAME = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(tokenizer.tokenize(output["sentence"][105])) # 기존 토크나이저
tokenizer.add_special_tokens({"additional_special_tokens": list(TYPE_TOKENS.values())})
print(tokenizer.tokenize(output["sentence"][105])) # special tokens 추가

['<', 'o', '##b', '##j', ':', 'PO', '##H', '>', '첼', '##리스트', '<', '/', 'o', '##b', '##j', ':', 'PO', '##H', '>', '<', 'sub', '##j', ':', 'PER', '>', '정명', '##화', '<', '/', 'sub', '##j', ':', 'PER', '>', ',', '바이올리니스트', '정경', '##화', '##의', '동생', '##이다']
['<obj:POH>', '첼', '##리스트', '</obj:POH>', '<subj:PER>', '정명', '##화', '</subj:PER>', ',', '바이올리니스트', '정경', '##화', '##의', '동생', '##이다']


special token을 추가하여 토크나이징 된 것을 확인할 수 있습니다.